# Final merge of the road and bridge datsets

In [53]:
import pandas as pd
import numpy as np
from haversine import haversine, Unit

In [54]:
documented_intersections_start_end = pd.read_csv('../data/semi-processed/identified_roads_final.csv')
roads_in_simulation = documented_intersections_start_end.road.unique().tolist()
len(roads_in_simulation)

26

In [55]:
documented_intersections_start_end

,Unnamed: 0,road,chainage,LRPName,lat,lon,gap,type,name,road_to,distance_to_closest,closest_lat,closest_lon,closest_lrp,closest_name,closest_type,closest_chainage,intersection,trios,model_type
0,0,N1,0.000,LRPS,23.706028,90.443333,NaN,Others,Start of Road after Jatrabari Flyover infront...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,group 1,sourcesink
1,55,N1,8.763,LRP009a,23.706083,90.521527,NaN,"sideroad,left",Road to Sylhet (N2),N2,20.374160,23.705917,90.521444,LRPS,Road Start from N1 at Katchpur,Others,0.000,intersection,group 19,sourcesink
2,56,N1,45.322,LRP046a,23.530749,90.781610,NaN,"sideroad,right","Z1402, road to Pennai",Z1402,0.000000,23.530749,90.781610,LRPE,Meet with N1 at Pennai,Others,43.849,intersection,group 19,intersourcesink
3,57,N1,47.224,LRP047a,23.530694,90.792444,NaN,"sideroad,right",Z1044,Z1044,0.000000,23.530694,90.792444,LRPS,Start of Road From N1 at Gouripur,Others,0.000,intersection,group 20,sourcesink
4,58,N1,55.477,LRP056c,23.512667,90.871416,NaN,"sideroad,left","Z1042, road to Bancharampur",Z1042,3.091223,23.512639,90.871416,LRPS,Start of Road from N1 at Eliotganj.,Others,0.000,intersection,group 20,sourcesink
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,49,Z1402,43.849,LRPE,23.530749,90.781610,NaN,Others,Meet with N1 at Pennai,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,group 17,sourcesink
110,87,Z1402,45.322,LRPE,23.530749,90.781610,NaN,Others,Meet with N1 at Pennai,N1,0.000000,23.530749,90.781610,LRP046a,NaN,NaN,43.849,NaN,NaN,intersection
111,50,Z2013,0.000,LRPS,24.867305,91.856806,NaN,Others,Road Start From N2 at Chandipool Sylhet,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,group 17,intersourcesink
112,51,Z2013,25.045,LRPE,24.657222,91.828222,NaN,Others,Road End With Borobaga River.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,group 18,sourcesink


In [56]:
documented_intersections_start_end[['model_type','road_to','road']]

,model_type,road_to,road
0,sourcesink,NaN,N1
1,sourcesink,N2,N1
2,intersourcesink,Z1402,N1
3,sourcesink,Z1044,N1
4,sourcesink,Z1042,N1
...,...,...,...
109,sourcesink,NaN,Z1402
110,intersection,N1,Z1402
111,intersourcesink,NaN,Z2013
112,sourcesink,NaN,Z2013


In [57]:
documented_intersections_start_end.model_type.value_counts()

model_type
intersection       48
sourcesink         44
intersourcesink    22
Name: count, dtype: int64

In [58]:
documented_intersections_start_end.model_type.value_counts()

model_type
intersection       48
sourcesink         44
intersourcesink    22
Name: count, dtype: int64

In [59]:
bridges = pd.read_csv('../data/semi-processed/bridges_vul_index.csv')
bridges['model_type']='bridge'
bridges.drop(columns=['Unnamed: 0'], inplace=True)
# This is to filter out the cleaning we did at the distance to closest< threshold which was not done with the list used for the bridges
bridges = bridges[bridges['road'].isin(roads_in_simulation)]
bridges

,Unnamed: 0.1,road,km,type,LRPName,name,length,condition,structureNr,roadName,...,zone,circle,division,sub-division,lat,lon,EstimatedLoc,lanes,vul_cat,model_type
0,0,R250,8.107,Box Culvert,LRP008a,HELAL PUR BOX CULVERT,4.50,A,100489,Sylhet-Golapganj-Charkhai-Jakiganj Road,...,Sylhet,Sylhet,Sylhet,Golapganj,24.852237,91.934565,interpolate,2.0,2.0,bridge
1,1,R250,12.648,Box Culvert,LRP013a,SUKNA BOX CULVERT,3.00,A,100487,Sylhet-Golapganj-Charkhai-Jakiganj Road,...,Sylhet,Sylhet,Sylhet,Golapganj,24.853910,91.976418,interpolate,2.0,2.0,bridge
2,2,R250,15.317,Box Culvert,LRP015b,FULBARI BOX CULVERT,3.00,A,100491,Sylhet-Golapganj-Charkhai-Jakiganj Road,...,Sylhet,Sylhet,Sylhet,Golapganj,24.854289,92.001478,interpolate,2.0,2.0,bridge
3,3,R250,14.563,Box Culvert,LRP015a,FULBARI BOX CULVERT,3.00,A,100490,Sylhet-Golapganj-Charkhai-Jakiganj Road,...,Sylhet,Sylhet,Sylhet,Golapganj,24.856098,91.994234,interpolate,2.0,2.0,bridge
4,4,R250,14.032,Box Culvert,LRP014a,BOYTIKOL BOX CULVERT,3.00,A,100488,Sylhet-Golapganj-Charkhai-Jakiganj Road,...,Sylhet,Sylhet,Sylhet,Golapganj,24.857246,91.989462,interpolate,2.0,2.0,bridge
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2380,2380,N204,29.610,Box Culvert,LRP035c,CRIKOTA,1.55,A,104081,Jagadishpur-Chunarughat-Shaistaganj (old part),...,Sylhet,Moulavi Bazar,Habiganj,Shaistagonj,24.232763,91.504116,interpolate,2.0,2.0,bridge
2381,2381,N204,31.801,Box Culvert,LRP038a,DURGAPUR,1.50,A,104084,Jagadishpur-Chunarughat-Shaistaganj (old part),...,Sylhet,Moulavi Bazar,Habiganj,Shaistagonj,24.249896,91.493845,interpolate,2.0,2.0,bridge
2382,2382,N204,32.210,RCC Girder Bridge,LRP038b,DORGAPUR RCC GIDER BRIDGE,17.80,C,104082,Jagadishpur-Chunarughat-Shaistaganj (old part),...,Sylhet,Moulavi Bazar,Habiganj,Shaistagonj,24.252372,91.491274,interpolate,2.0,2.0,bridge
2383,2383,N204,33.550,Box Culvert,LRP040a,NEAR SHAIS TAGAMJ BOX CULVERT,4.50,A,104077,Jagadishpur-Chunarughat-Shaistaganj (old part),...,Sylhet,Moulavi Bazar,Habiganj,Shaistagonj,24.261595,91.482321,interpolate,2.0,2.0,bridge


In [60]:
data_points = pd.concat([bridges, documented_intersections_start_end], ignore_index=True)
data_points.sort_values(by=['road','chainage'], inplace=True)
data_points

,Unnamed: 0.1,road,km,type,LRPName,name,length,condition,structureNr,roadName,...,road_to,distance_to_closest,closest_lat,closest_lon,closest_lrp,closest_name,closest_type,closest_chainage,intersection,trios
2385,NaN,N1,NaN,Others,LRPS,Start of Road after Jatrabari Flyover infront...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,group 1
2000,2000.0,N1,1.800,Box Culvert,LRP001a,.,11.3,A,117861.0,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998,1998.0,N1,4.925,Box Culvert,LRP004b,.,6.6,A,117862.0,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2386,NaN,N1,NaN,"sideroad,left",LRP009a,Road to Sylhet (N2),NaN,NaN,NaN,NaN,...,N2,20.37416,23.705917,90.521444,LRPS,Road Start from N1 at Katchpur,Others,0.0,intersection,group 19
2001,2001.0,N1,8.976,PC Girder Bridge,LRP008b,KANCHPUR PC GIRDER BRIDGE,397.0,C,101102.0,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2080,2080.0,Z2013,18.148,Box Culvert,LRP018a,AZIZADUR,6.0,A,105443.0,Sylhet (Telikhal)-Sultanpur-Balaganj Road,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2079,2079.0,Z2013,18.732,Box Culvert,LRP018b,LONGIRA CULVERT,3.0,A,105445.0,Sylhet (Telikhal)-Sultanpur-Balaganj Road,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2078,2078.0,Z2013,18.995,Box Culvert,LRP018c,LONGIRA CULVERT,6.0,A,105447.0,Sylhet (Telikhal)-Sultanpur-Balaganj Road,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2497,NaN,Z2013,NaN,Others,LRPE,Road End With Borobaga River.,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,group 18


## Now, lets add the inbetween links and calculate the length of the links

In [61]:
def add_links_rows(df):
    nans = np.where(np.empty_like(df.values), np.nan, np.nan)    
    data = np.hstack([df.values, nans]).reshape(-1, df.shape[1])
    new_df = pd.DataFrame(data, columns=df.columns)
    return new_df.iloc[:-1, :]

N1N2_empty_links = add_links_rows(data_points)
N1N2_empty_links

,Unnamed: 0.1,road,km,type,LRPName,name,length,condition,structureNr,roadName,...,road_to,distance_to_closest,closest_lat,closest_lon,closest_lrp,closest_name,closest_type,closest_chainage,intersection,trios
0,NaN,N1,NaN,Others,LRPS,Start of Road after Jatrabari Flyover infront...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,group 1
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000.0,N1,1.8,Box Culvert,LRP001a,.,11.3,A,117861.0,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1998.0,N1,4.925,Box Culvert,LRP004b,.,6.6,A,117862.0,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4992,2078.0,Z2013,18.995,Box Culvert,LRP018c,LONGIRA CULVERT,6.0,A,105447.0,Sylhet (Telikhal)-Sultanpur-Balaganj Road,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4994,NaN,Z2013,NaN,Others,LRPE,Road End With Borobaga River.,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,group 18
4995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
def fill_links(df):
    countlink = 1
    countlrp = 1
    ab = 0
    df['road'].bfill( inplace=True)
    for index in range(len(df)):
        ## FILLING THE LINKS
        if pd.isna(df.loc[index,'LRPName']):
            
            df.at[index, 'length'] = haversine((df.at[(index-1), 'lat'], df.at[(index-1), 'lon']), (df.at[(index+1), 'lat'], df.at[(index+1), 'lon']), unit=Unit.METERS)
            
            df.at[index, 'lat'] = df.at[(index+1), 'lat']+ (df.at[(index+1), 'lat']-df.at[(index-1), 'lat'])/2
            df.at[index, 'lon'] = df.at[(index+1), 'lon']+ (df.at[(index+1), 'lon']-df.at[(index-1), 'lon'])/2
            df.at[index, 'model_type'] ='link'
            
            ## NAMING THE LINKS INBETWEEN
            if index + 2< len(df):
                ## IF YOUR PREVIOUS IS A SOURCE, OR YOUR NEXT IS A SOURCE, THEN 
                if ((df.at[index-1, 'model_type'] =='sourcesink') and (df.at[index+1, 'model_type'] =='sourcesink')):
                    ## IF YOU HAVE A NEGATIVE LENGTH, THEN YOU ARE INBETWEEN TWO DIFFERENT ROADS. SO YOU DONT EXIST!! THATS WHY MODEL TYPE GOES TO "not a real link"
                    df.at[index, 'name'] ='not a real link'
                    countlink = 1
                
                # ## IF YOUR PREVIOUS OR NEXT IS A SOURCE, HIDDEN DUE TO DUPLICATES (INTERSECTION& SOURCESINK)   
                # elif (df.at[index, 'length']==0) and (df.at[index-1, 'lat']==df.at[index+1, 'lat']) and (df.at[index-1, 'lon']==df.at[index+1, 'lon']): 
                #      df.at[index, 'name'] ='not a real link'
                #      countlink = 1
                    
                ## IF THE ROAD OF THE NEXT LINK IS THE SAME AS YOUR ROAD, AND YOU HAVE A POSITIVE LINK
                ##THEN NAME YOURSELF THE NEXT AVAILABLE NAME
                elif df.loc[(index+2),'road']==df.loc[index,'road']:
                    df.at[index, 'name'] ='link'+' '+str(countlink)
                    countlink +=1
                ## IF THE NEXT LINK IS ON ANOTHER ROAD THEN NAME YOURSELF AND RESET THE COUNTER SO THE NEXT LINK WILL BE "link 1" OF HIS OWN ROAD
                elif df.loc[(index+2),'road']!=df.loc[index,'road']:
                    df.at[index, 'name'] ='link'+' '+str(countlink)
                    countlink = 1
        
        elif index + 2< len(df):
            if (df.loc[index,'model_type']=='model_type'):
                ab+=1
                df.at[index, 'length'] = haversine(
                    (df.at[(index-1), 'lat'], df.at[(index-1), 'lon']), 
                    (df.at[(index+1), 'lat'], df.at[(index+1), 'lon']), unit=Unit.METERS)
            
                    
        ## FILLING THE BRIGDES        
        else:
            # Naming LRPS in the dataset
            if index + 2< len(df):
                ## If the road of the next LRP is still the same and the bridge length is positive, name yourself the next available name and counter+1
                if (df.loc[(index+2),'road']==df.loc[index,'road']):
                    name ='bridge'+' '+str(countlrp)
                    countlrp +=1
                ## If the next LRP is on a different road, then name yourself and reset the counter after
                elif (df.loc[(index+2),'road']!=df.loc[index,'road']):
                    name ='bridge'+' '+str(countlrp)
                    countlrp = 1
            # 
            if df.at[index,'name']=='.' or df.at[index, 'name']==0:
                df.at[index, 'name']= name
        
    return df
    
N1N2_filled_links = fill_links(N1N2_empty_links)
N1N2_filled_links

,Unnamed: 0.1,road,km,type,LRPName,name,length,condition,structureNr,roadName,...,road_to,distance_to_closest,closest_lat,closest_lon,closest_lrp,closest_name,closest_type,closest_chainage,intersection,trios
0,NaN,N1,NaN,Others,LRPS,Start of Road after Jatrabari Flyover infront...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,group 1
1,NaN,N1,NaN,NaN,NaN,link 1,1776.650373,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000.0,N1,1.8,Box Culvert,LRP001a,.,11.3,A,117861.0,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,N1,NaN,NaN,NaN,link 2,2978.663177,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1998.0,N1,4.925,Box Culvert,LRP004b,.,6.6,A,117862.0,Dhaka (Jatrabari)-Comilla (Mainamati)-Chittago...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4992,2078.0,Z2013,18.995,Box Culvert,LRP018c,LONGIRA CULVERT,6.0,A,105447.0,Sylhet (Telikhal)-Sultanpur-Balaganj Road,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4993,NaN,Z2013,NaN,NaN,NaN,link 25,5331.997408,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4994,NaN,Z2013,NaN,Others,LRPE,Road End With Borobaga River.,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,group 18
4995,NaN,Z2013,NaN,NaN,NaN,NaN,23537.780069,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [63]:
fakes = N1N2_filled_links[N1N2_filled_links['name']=='not a real link']
fakes

,Unnamed: 0.1,road,km,type,LRPName,name,length,condition,structureNr,roadName,...,road_to,distance_to_closest,closest_lat,closest_lon,closest_lrp,closest_name,closest_type,closest_chainage,intersection,trios
1661,NaN,N2,NaN,NaN,NaN,not a real link,35625.764971,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3049,NaN,R151,NaN,NaN,NaN,not a real link,44396.756194,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3517,NaN,R220,NaN,NaN,NaN,not a real link,17206.600537,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4259,NaN,Z1005,NaN,NaN,NaN,not a real link,282923.920702,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
fake_links_roads = fakes.road.to_list()
fake_links_roads

['N2', 'R151', 'R220', 'Z1005']

In [65]:
manually_check_start_end = []
for i in roads_in_simulation:
    if i not in fake_links_roads:
        manually_check_start_end.append(i)
        
len(manually_check_start_end)

22

In [66]:
## Visual inspection
N1N2_filled_links[2800:][['road','road_to','model_type','lat','lon','length','chainage']]

,road,road_to,model_type,lat,lon,length,chainage
2800,N207,NaN,bridge,24.472502,91.765472,18.45,43.036
2801,N207,NaN,link,24.540001,91.710776,6217.003646,NaN
2802,N207,NaN,bridge,24.517502,91.729008,4.55,51.25
2803,N207,NaN,link,24.522498,91.731362,402.996007,NaN
2804,N207,NaN,bridge,24.520833,91.730578,9.2,51.684
...,...,...,...,...,...,...,...
4992,Z2013,NaN,bridge,24.704475,91.837201,6.0,18.995
4993,Z2013,NaN,link,24.633596,91.823732,5331.997408,NaN
4994,Z2013,NaN,sourcesink,24.657222,91.828222,NaN,25.045
4995,Z2013,NaN,link,24.972346,91.871098,23537.780069,NaN


In [67]:
fakes2 = [1479, 1565, 2695, 2837, 3157, 3345, 3586, 3725, 3777, 3935, 3999, 4367, 4445, 4509, 4589, 4717, 4825, 4945]

In [68]:
fake_links = fakes.index.tolist() + fakes2


In [69]:
for i in fake_links:
    N1N2_filled_links.drop(i, inplace=True)

In [70]:
N1N2_filled_links[N1N2_filled_links['model_type']=='intersection'][['model_type','road','LRPName','road_to','closest_lrp']]

,model_type,road,LRPName,road_to,closest_lrp
148,intersection,N1,LRP094,R140,LRP004a
150,intersection,N1,LRP094a,R140,LRP004a
692,intersection,N1,LRP341b,Z1005,LRPS
1474,intersection,N102,LRPS,N1,LRP084a
1478,intersection,N102,LRPE,N2,LRP086a
1564,intersection,N104,LRP001a,N1,LRP148a
1580,intersection,N105,LRP012a,N2,LRP012a
1666,intersection,N2,LRPS,N1,LRP009a
1672,intersection,N2,LRP012d,R203,LRPS
1710,intersection,N2,LRP030b,R301,LRPE


In [71]:
id=1
for i in N1N2_filled_links.index:
    N1N2_filled_links.at[i,'id']= id
    id+=1

ids_dict = {}        
for i in N1N2_filled_links[N1N2_filled_links['model_type']=='intersection'].index:
    if (N1N2_filled_links.at[i,'road']=='N1') or (N1N2_filled_links.at[i,'road']=='N2'):
        ids_dict[N1N2_filled_links.at[i,'id']]= (N1N2_filled_links.at[i,'lat'],N1N2_filled_links.at[i,'lon'])   

In [72]:
ct = 0
for index, row in N1N2_filled_links.iterrows():
    lat_lon_pair = (row['lat'], row['lon'])
    for key, value_set in ids_dict.items():
        # print(type(value_set))
        if (lat_lon_pair[0]==value_set[0]) and (lat_lon_pair[1]==value_set[1]):
            ct +=1
            N1N2_filled_links.at[index, 'id'] = key
            break  # Exit the loop once a match is found

In [73]:
ct

51

In [74]:
N1N2_filled_links = N1N2_filled_links.reset_index()
N1N2_filled_links

,index,Unnamed: 0.1,road,km,type,LRPName,name,length,condition,structureNr,...,distance_to_closest,closest_lat,closest_lon,closest_lrp,closest_name,closest_type,closest_chainage,intersection,trios,id
0,0,NaN,N1,NaN,Others,LRPS,Start of Road after Jatrabari Flyover infront...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,group 1,1.0
1,1,NaN,N1,NaN,NaN,NaN,link 1,1776.650373,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
2,2,2000.0,N1,1.8,Box Culvert,LRP001a,.,11.3,A,117861.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0
3,3,NaN,N1,NaN,NaN,NaN,link 2,2978.663177,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
4,4,1998.0,N1,4.925,Box Culvert,LRP004b,.,6.6,A,117862.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4970,4992,2078.0,Z2013,18.995,Box Culvert,LRP018c,LONGIRA CULVERT,6.0,A,105447.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4971.0
4971,4993,NaN,Z2013,NaN,NaN,NaN,link 25,5331.997408,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4972.0
4972,4994,NaN,Z2013,NaN,Others,LRPE,Road End With Borobaga River.,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,group 18,4973.0
4973,4995,NaN,Z2013,NaN,NaN,NaN,NaN,23537.780069,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4974.0


In [75]:
for i in N1N2_filled_links.index:
    if i+2 <= len(N1N2_filled_links):
        if N1N2_filled_links.at[i+1, 'road']!= N1N2_filled_links.at[i, 'road']:
            N1N2_filled_links.at[i, 'length']= 0

In [76]:
N1N2_filled_links.model_type.value_counts()

model_type
link               2477
bridge             2385
intersection         47
sourcesink           44
intersourcesink      22
Name: count, dtype: int64

In [77]:
N1N2_filled_links.to_csv('../data/processed/cleaned_data_final.csv')